In [ ]:
! git clone https://github.com/joeynmt/joeynmt.git

Cloning into 'joeynmt'...
remote: Enumerating objects: 3116, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 3116 (delta 94), reused 135 (delta 86), pack-reused 2951
Receiving objects: 100% (3116/3116), 8.09 MiB | 22.14 MiB/s, done.
Resolving deltas: 100% (2123/2123), done.


In [ ]:
% cd joeynmt/

/content/joeynmt


In [ ]:
!pip3 install .

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
# create config file for JoeyNMT
import time
from datetime import datetime
t = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

name= 'enro'
date= t

#gdrive_path= os.environ["gdrive_path"]

#create the config
config = """
name: "{name}_transformer"


data:
    src: "en"
    trg: "ro"
    train: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_train.bpe"
    dev:   "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_dev.bpe"
    test:  "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_vocab.txt"
    trg_vocab: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/drive/MyDrive/Colab Notebooks/BA_Colab_Notebooks/models/enro_dcep_transformer_2021-05-16 18:43:22/best.ckpt"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999]
    scheduling: "plateau"
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100
    validation_freq: 1000
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "/content/drive/MyDrive/Colab\ Notebooks/BA_Colab_Notebooks/models/{name}_dcep_transformer_{date}/"
    #model_dir: "/content/drive/MyDrive/Colab Notebooks/BA_Colab_Notebooks/models/enro_dcep_transformer_1"
    overwrite: False
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 5
    save_latest_ckpt: True

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.1
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.1



""".format(name=name, date=date)
with open("/content/joeynmt/configs/transformer_enro_dcep.yaml", 'w') as f:
  f.write(config)

In [ ]:
# create config file for JoeyNMT
import time
from datetime import datetime
t = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

name= 'enro'
date= t

#gdrive_path= os.environ["gdrive_path"]

#create the config
config = """
name: "bahdanau_dcep_enro"

data:
    src: "en"
    trg: "ro"
    train: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_train.bpe"
    dev:   "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_dev.bpe"
    test:  "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_vocab.txt"
    trg_vocab: "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_vocab.txt"
   
testing:
    beam_size: 10
    alpha: 1.0

training:
    random_seed: 42
    optimizer: "adam"
    learning_rate: 0.001
    learning_rate_min: 0.00005
    batch_size: 64
    scheduling: "plateau"
    patience: 0
    decrease_factor: 0.5
    early_stopping_metric: "ppl"
    epochs: 100
    validation_freq: 2359
    logging_freq: 500
    eval_metric: "bleu"
    model_dir: "/content/drive/MyDrive/Colab\ Notebooks/BA_Colab_Notebooks/models/{name}_{date}/"
    overwrite: False
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2]

model:
    encoder:
        rnn_type: "lstm"
        embeddings: 
            embedding_dim: 256
            scale: False
        hidden_size: 256
        bidirectional: True
        dropout: 0.0
        num_layers: 1
    decoder:
        rnn_type: "lstm"
        embeddings:
            embedding_dim: 256
            scale: False
        emb_scale: False
        hidden_size: 256
        dropout: 0.2
        hidden_dropout: 0.2
        num_layers: 1
        input_feeding: True
        init_hidden: "last"
        attention: "luong"


""".format(name=name, date=date)
with open("/content/joeynmt/configs/bahdanau_enro_dcep.yaml", 'w') as f:
  f.write(config)

In [ ]:
!python -m joeynmt train /content/joeynmt/configs/bahdanau_enro_dcep.yaml

2021-05-20 16:04:02,391 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-20 16:04:02,422 - INFO - joeynmt.data - Loading training data...
2021-05-20 16:04:18,446 - INFO - joeynmt.data - Building vocabulary...
2021-05-20 16:04:46,116 - INFO - joeynmt.data - Loading dev data...
2021-05-20 16:04:46,353 - INFO - joeynmt.data - Loading test data...
2021-05-20 16:04:50,898 - INFO - joeynmt.data - Data loaded.
2021-05-20 16:04:50,898 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-20 16:04:51,571 - INFO - joeynmt.model - Enc-dec model built.
2021-05-20 16:04:51.699420: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-20 16:04:53,067 - INFO - joeynmt.training - Total params: 32574208
2021-05-20 16:04:55,810 - INFO - joeynmt.helpers - cfg.name                           : bahdanau_dcep_enro
2021-05-20 16:04:55,811 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021

In [ ]:
!head -n 30 /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_dev.bpe.ro

/ oeil / Fin@@ d@@ B@@ y@@ Proc@@ num@@ .do ? lang = 2 &amp; proc@@ n@@ um = COD / 2008 / 0221
Articolul 3 litera ( id ) ( nouă )
fezabilitatea depozit@@ ării geologice
13 . ’
&quot; ( 7 ) Prin urmare , măsurile prevăzute de prezentul regulament trebuie să faciliteze comercializarea blăn@@ urilor şi a articolelor din blană provenind de la alte specii decât pisi@@ ci şi câ@@ ini şi să pre@@ întâmp@@ ine , în general , orice perturb@@ are a pieţei interne de blănuri şi articole din blană . &quot;
&quot; având în vedere rezoluția sa din 5 iunie 2008 privind punerea în aplicare a Strategiei europene de securitate și a PESA Texte adoptate , P6 _ TA ( 2008 ) 0255 . &quot;
&quot; exploatează , între@@ țin și dezvoltă , în condiții economice , instalații de transport , depozitare și / sau de GNL sigure , fiabile și eficiente &quot;
&quot; subliniază faptul că revizuirea Strategiei Europene pentru ocuparea forţei de muncă ( SE@@ O@@ FM ) şi orientările integrate pentru următorul ciclu ( 2008-20

In [ ]:
!python3 -m joeynmt test /content/joeynmt/configs/transformer_enro_dcep.yaml --output_path "/content/drive/MyDrive/Colab Notebooks/BA_Colab_Notebooks/models/enro_dcep_transformer_2021-05-16 18:43:22/out."


Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/joeynmt/joeynmt/__main__.py", line 3, in <module>
    from joeynmt.training import train
  File "/content/joeynmt/joeynmt/training.py", line 17, in <module>
    import torch
  File "/usr/local/lib/python3.7/dist-packages/torch/__init__.py", line 196, in <module>
    from torch._C import *
RuntimeError: KeyboardInterrupt: 


In [ ]:
! head -n 10 /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_train.bpe.ro >> /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/very_small_clean_train.bpe.ro

In [ ]:
!wc -l /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/very_small_clean_train.bpe.ro

10 /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/very_small_clean_train.bpe.ro


In [ ]:
!python3 -m joeynmt translate /content/joeynmt/configs/transformer_enro_dcep.yaml < /content/drive/MyDrive/data_en_ro/DCEP_EN_RO/very_small_clean_train.bpe.ro> out.

2021-05-17 10:53:16,340 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-17 10:53:43,170 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-17 10:53:44,461 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!ls

benchmarks.md	    joey_demo.ipynb  out.	       setup.py
CODE_OF_CONDUCT.md  joeynmt	     README.md	       small_clean_train.bpe.ro
configs		    joey-small.png   requirements.txt  test
docs		    LICENSE	     scripts




*   detokenize output
*   tensorboard
*   language detection



In [ ]:
#!head -n 1 "/content/drive/MyDrive/Colab Notebooks/BA_Colab_Notebooks/models/enro_dcep_transformer_2021-05-16 06:54:30/out..dev"
!tail -n 30 "/content/drive/MyDrive/data_en_ro/DCEP_EN_RO/clean_dcep.ro-en.ro.txt"


"solicită VP/ÎR să se asigure că guvernul din Bahrain își respectă promisiunile de a respecta drepturile omului, de a implementa reformele necesare, de a iniția anchete independente ale cazurilor de încălcare a drepturilor omului și de a se asigura că cei responsabili sunt trași la răspundere; îndeamnă guvernul din Bahrain să renunțe la acuzațiile împotriva medicilor și cadrelor medicale și să-i elibereze pe toți cei aflați în arest pentru participarea la proteste pașnice pro-democrație;"
"solicită autorităților din Bahrain să reinstaureze și să respecte drepturile omului și libertățile fundamentale, inclusiv libertatea de exprimare, atât online, cât și offline, libertatea de întrunire, libertatea religioasă, drepturile femeilor și egalitatea de gen, pentru a combate discriminarea și pentru a pune capăt tuturor restricțiilor în ceea ce privește accesul la tehnologiile de informare și comunicare; solicită autorităților din Bahrain să suspende toate restricțiile în ceea ce privește acces

In [ ]:
#%load_ext tensorboard
#! tensorboard --logdir model_dir/tensorboard
!tensorboard dev upload \
  --logdir '/content/drive/MyDrive/Colab Notebooks/BA_Colab_Notebooks/models/enro_dcep_transformer_1' \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \


2021-05-16 09:56:09.617974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/nY5ZM9OLT6Kb0soKLO7lmA/

[2021-05-16T09:56:11] Started scanning logdir.
[2021-05-16T09:56:12] Total uploaded: 19 scalars, 0 tensors, 0 binary objects


In [ ]:
! python3 /content/joeynmt/scripts/plot_validations.py /content/drive/MyDrive/Colab\ Notebooks/BA_Colab_Notebooks/models/enro_transformer_2/ --plot_values bleu PPL --output_path /content/drive/MyDrive/Colab\ Notebooks/BA_Colab_Notebooks/models/enro_transformer_2/my_plot.pdf